In [1]:
import os
import sys
import platform

if platform.system() == "Windows":
    sys.path.append('C:/Users/WORKSTATION2/Self_Driving_Car/')
    sys.path.append('C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/')
    sys.path.append('C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/model')
else: # Ubunut
    sys.path.append('/home/user/Self_Driving_Car/')
    sys.path.append('/home/user/Self_Driving_Car/Imitation_Learning_RL/')
    sys.path.append('/home/user/Self_Driving_Car/Imitation_Learning_RL/model')


In [2]:
import torch
from stable_baselines3 import SAC
from carla_gym_env import CarlaEnv
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from custom_feature_extractor import UnifiedFeatureExtractor
from stable_baselines3.common.logger import TensorBoardOutputFormat, Logger
from stable_baselines3.common.callbacks import CheckpointCallback, CallbackList

['c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\python310.zip', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\DLLs', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15', '', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages\\win32', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages\\win32\\lib', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages\\Pythonwin', 'C:/Users/WORKSTATION2/Self_Driving_Car/', 'C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/', 'C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/model', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL/model', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL/model']


c:\Users\WORKSTATION2\Miniconda3\envs\carla_15\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\WORKSTATION2\Miniconda3\envs\carla_15\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
CHECKPOINTS_DIR = "checkpoints"
TENSORBOARD_LOGS = "tensorboard_logs"
CHECKPOINT_PATH = os.path.join(CHECKPOINTS_DIR, "sac_carla_model")
TOTAL_TIMESTEPS = 5_000_000
SAVE_INTERVAL = 100_000
DEVICE = "cuda" if torch.cuda.is_available() else 'cpu'

In [4]:
def make_env(host, port):
    def _init():
        return CarlaEnv(host=host, port=port)
    return _init

In [5]:
env_fns = [
    make_env("localhost", 2000),
    make_env("localhost", 2001),
    make_env("192.168.0.2", 2003),
    make_env("192.168.0.2", 2006)
    # make_env("127.0.0.1", 2002)
]

In [ ]:
def main():
    os.makedirs(CHECKPOINTS_DIR, exist_ok=True)
    os.makedirs(TENSORBOARD_LOGS, exist_ok=True)
    
    # try:
    #     env=SubprocVecEnv(env_fns)
    # except Exception as e:
    #     print(f"Failed to create multi-envs: {e}")
    #     env=DummyVecEnv([make_env("localhost", 2000)])

    env=SubprocVecEnv(env_fns)
    

    print(env.observation_space)

    
    policy_kwargs = {
        'features_extractor_class' : UnifiedFeatureExtractor,

        'features_extractor_kwargs' : {"features_dim" : 772},
        

        # 'net_arch' : dict(pi=[768, 256, 256], qf=[768, 256, 256])
        'net_arch' : dict(pi=[512, 256], qf=[512, 256])
        
    }
    
    if os.path.exists(CHECKPOINT_PATH + ".zip"):
        model = SAC.load(CHECKPOINT_PATH, env=env, device=DEVICE)
        print(f"Resuming training from checkpoint: {CHECKPOINT_PATH}")
    else:
        print("Starting new training.")
        
        model = SAC(
            "MultiInputPolicy",
            env,
            policy_kwargs=policy_kwargs,
            learning_rate=3e-4,
            buffer_size=5000,
            learning_starts=10000,
            batch_size=256,
            tau=0.005,
            gamma=0.99,
            train_freq=4,
            gradient_steps=2,
            tensorboard_log=TENSORBOARD_LOGS,
            verbose=1,
            device=DEVICE
        )
    
    checkpoint_callback = CheckpointCallback(save_freq=SAVE_INTERVAL, save_path=CHECKPOINTS_DIR, name_prefix='sac_carla_model')
    # tensorboard_callback = TensorBoardOutputFormat(folder=TENSORBOARD_LOGS)
    # callbacks = CallbackList([checkpoint_callback, tensorboard_callback])
    callbacks = CallbackList([checkpoint_callback])
    model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=callbacks, progress_bar=True)
    model.save("sac_carla_model")
    
    final_model_path = os.path.join(CHECKPOINTS_DIR, f"sac_final_{TOTAL_TIMESTEPS//1000}k")
    model.save(final_model_path)
    print(f"[DONE] Final model saved to: {final_model_path}")


In [7]:
if __name__ == "__main__":
    main()

Failed to create multi-envs: 


RuntimeError: time-out of 10000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000